# **Metabolism Analysis of T. muris and C. elegans**

---

#### **Students Names:** Abril Iglesias Jiménez, Patricia Sánchez Mengual, Carla Licer Otero, Laia Montenegro Domenech, Esther Batalla Royo and Andrea Lauro
#### **Date:** [Fecha de entrega]
#### **Course:**  Computational Biology and Biomedical Data Analysis - Universitat Rovira i Virgili

---
## **1. Introduction**

Soil-transmitted helminth infections cause health problems such as diarrhea, abdominal pain, malnutrition, and growth retardation. Human trichuriasis, caused by *Trichuris trichiura*, is one of the most common infections.

Due to the difficulty of studying this species directly in the laboratory, *Trichuris muris*, a mouse parasite with very similar biological and pathological characteristics, is used as an experimental model.

The main objective of this project is to analyze the metabolism of *T. muris*, compare it with *Caenorhabditis elegans*, and identify essential genes that could serve as potential therapeutic targets for the treatment of whipworm infections.

## **2. Genome-Scale Metabolic Models Acquisition**

In this section, we will obtain and load the genome-scale metabolic models (GEMs) of *Trichuris muris* and *Caenorhabditis elegans* from public GitHub repositories.  


## Import libraries

In [37]:
import cobra
import gzip
from cobra.io import load_model
import threading
from Bio import SeqIO, Align
from Bio.Align import PairwiseAligner
import pandas as pd
import time
from joblib import Parallel, delayed
from itertools import chain
import warnings
from Bio import BiopythonDeprecationWarning


## Set file paths

In [38]:
tmuris_xml = '/opt/notebooks/final_project/Genomicscale/Tmuris.xml'
celegans_xml = '/opt/notebooks/final_project/Genomicscale/Celegans.xml'
tmuris_protein_fa = '/opt/notebooks/final_project/AAsequence/Tmuris.protein.fa.gz'
celegans_protein_fa = '/opt/notebooks/final_project/AAsequence/Celegans.protein.fa.gz'

In [39]:
# Load the genome-scale metabolic model of T. muris
T_muris_GS = cobra.io.read_sbml_model(tmuris_xml)

# Load the genome-scale metabolic model of C. elegans
C_elegans_GS = cobra.io.read_sbml_model(celegans_xml)

display(T_muris_GS)
display(C_elegans_GS)

https://identifiers.org/taxonomy/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id


Name,iTMU798
Memory address,7c8c68a88cb0
Number of metabolites,930
Number of reactions,1219
Number of genes,800
Number of groups,0
Objective expression,1.0*WBPTMR0091 - 1.0*WBPTMR0091_reverse_cd110
Compartments,"Mitochondria, Cytosol, Extracellular Space"


Name,WormGEM
Memory address,7c8c6895f1d0
Number of metabolites,8175
Number of reactions,11936
Number of genes,1604
Number of groups,153
Objective expression,1.0*MAR00021 - 1.0*MAR00021_reverse_97974
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


The genome-scale model **iTMU798** of *T. muris* contains **930 metabolites**, **1219 reactions**, and **800 genes**, which aligns well with the original publication. The model includes three compartments (cytosol, mitochondria, and extracellular space). 

The genome-scale model **WormGEM** of *C. elegans* includes **8175 metabolites**, **11,936 reactions**, and **1604 genes**. It defines **153 metabolic groups**. The model icludes multiple cellular compartments, such as cytosol, nucleus, mitochondria, lysosome, endoplasmic reticulum, and others.

## **3. Protein Sequences Acquisition**

In this section, we obtain and load the amino acid sequences for all proteins from *Trichuris muris* and *Caenorhabditis elegans*.  
The sequences are retrieved from WormBase Parasite in FASTA format (.fa.gz files).


In [40]:
# Function to load protein sequences from a compressed .fa.gz file
def load_protein_sequences(file_path):
    with gzip.open(file_path, "rt") as handle:  # Open the .gz file in text mode
        records = list(SeqIO.parse(handle, "fasta"))  # Parse sequences in FASTA format
    return records  # Return the list of sequence records

# Load protein sequences for T. muris
t_muris_proteins = load_protein_sequences(tmuris_protein_fa)

# Load protein sequences for C. elegans
c_elegans_proteins = load_protein_sequences(celegans_protein_fa)

# Print the number of protein sequences loaded
print("T. muris proteins loaded", len(t_muris_proteins), " sequences.")
print("C. elegans proteins loaded:", len(c_elegans_proteins), "sequences.")
print("\n")

# Display information about the first T. muris protein
print("First T. muris protein:")
print("ID:", t_muris_proteins[0].id)  # Show protein ID
print("Description:", t_muris_proteins[0].description)  # Show description if available
print("Sequence:", t_muris_proteins[0].seq)  # Show the amino acid sequence

print("\n-------------------------------------------------------------------------------\n")

# Display information about the first C. elegans protein
print("First C. elegans protein:")
print("ID:", c_elegans_proteins[0].id)  
print("Description:", c_elegans_proteins[0].description)  
print("Sequence:", c_elegans_proteins[0].seq) 


T. muris proteins loaded 14995  sequences.
C. elegans proteins loaded: 28577 sequences.


First T. muris protein:
ID: TMUE_0000000001
Description: TMUE_0000000001 wormpep=TMP03343 gene=WBGene00295949 status=Predicted
Sequence: MVFPSFLLSSKTQRGIQKASKWWDESTKLAEAIVLGALGPSPATCNRCKELKIWWALFQPALPVAPSQ

-------------------------------------------------------------------------------

First C. elegans protein:
ID: 2L52.1a
Description: 2L52.1a wormpep=CE32090 gene=WBGene00007063 status=Confirmed uniprot=A4F336 insdc=CCD61130.1 product="C2H2-type domain-containing protein"
Sequence: MSMVRNVSNQSEKLEILSCKWVGCLKSTEVFKTVEKLLDHVTADHIPEVIVNDDGSEEVVCQWDCCEMGASRGNLQKKKEWMENHFKTRHVRKAKIFKCLIEDCPVVKSSSQEIETHLRISHPINPKKERLKEFKSSTDHIEPTQANRVWTIVNGEVQWKTPPRVKKKTVIYYDDGPRYVFPTGCARCNYDSDESELESDEFWSATEMSDNEEVYVNFRGMNCISTGKSASMVPSKRRNWPKRVKKRLSTQRNNQKTIRPPELNKNNIEIKDMNSNNLEERNREECIQPVSVEKNILHFEKFKSNQICIVRENNKFREGTRRRRKNSGESEDLKIHENFTEKRRPIRSCKQNISFYEMDGDIEEFEVFFDTPTKSKKVLLDIYSAKKMPKIEVEDSLVNKFHSKRPSRACRVLGSMEEVPFD

From these results, we can confirm that both proteomes were correctly loaded and are consistent with the expected data from WormBase Parasite. *T. muris* includes 14995 protein sequences, and *C. elegans* includes 28577. 


## **4. Identification of Orthologous Enzymes via Pairwise Sequence Alignment**

In this section, we focus on identifying orthologous enzymes between *Trichuris muris* and *Caenorhabditis elegans*.  We begin by extracting the protein-coding genes involved in enzymatic reactions from the *T. muris* metabolic model.

We then perform pairwise sequence alignment between each *T. muris* enzymatic protein and all proteins in the *C. elegans* to identify the best-matching orthologs based on sequence similarity.  
This analysis allows us to examine whether the metabolic function of each enzyme is conserved in *C. elegans*, or if there are significant differences that could indicate functional divergence or specialization.



In [51]:
#SEARCHING ID OF THE ENZYMATIC GENES

# Initialize an empty list to store enzymatic gene IDs
enzymatic_genes_tmuris =[]
    
# Loop through each gene object in the T. muris model, .genes gets the enzymatic proteins
for gene in T_muris_GS.genes:
    enzymatic_genes_tmuris.append(gene.id) #adds to a list the id of each gene
    
# Print results
print("Number of enzymatic genes in T. muris model: ", len(enzymatic_genes_tmuris))
print("Example gene IDs: ", enzymatic_genes_tmuris[:5])

# Crear el alineador UNA VEZ y configurar
aligner = PairwiseAligner()
aligner.mode = 'global'
aligner.match_score = match_score
aligner.mismatch_score = mismatch_score
aligner.open_gap_score = gap_open
aligner.extend_gap_score = gap_extend



output_file = "orthologs_blockwise.csv"




Number of enzymatic genes in T. muris model:  800
Example gene IDs:  ['TMUE_3000011849', 'TMUE_0000000186', 'TMUE_3000012793', 'TMUE_3000011199', 'TMUE_3000013102']


In [52]:
# Crear diccionario de secuencias de T. muris por ID
tmuris_seq_dict = {record.id: record for record in t_muris_proteins}

# Filtrar solo genes enzimáticos que tengan secuencia disponible
enzymatic_genes_with_seq = []
for gene_id in enzymatic_genes_tmuris:
    if gene_id in tmuris_seq_dict:
        enzymatic_genes_with_seq.append(tmuris_seq_dict[gene_id])

print(f"Genes enzimáticos de T. muris con secuencia disponible: {len(enzymatic_genes_with_seq)} / {len(enzymatic_genes_tmuris)}")


Genes enzimáticos de T. muris con secuencia disponible: 798 / 800


In [54]:

# Configuración previa
c_elegans_gene_ids = {gene.id for gene in C_elegans_GS.genes}
c_elegans_model_map = {gene.id: [rxn.id for rxn in gene.reactions] for gene in C_elegans_GS.genes}
min_len_ratio = 0.7
max_len_ratio = 1.4

aligner = Align.PairwiseAligner()
aligner.match_score = 1.0
aligner.mismatch_score = -1.0
aligner.open_gap_score = -1.5
aligner.extend_gap_score = -0.5

orthologs_detailed = []
total_genes = len(enzymatic_genes_with_seq)
print(f"Total de genes enzimáticos de T. muris a comparar: {total_genes}")

def comparar_con_todos(tm_protein):
    print(f"Comparando gen T. muris: {tm_protein.id}")
    with open("log.txt", "a") as f:
        f.write(f"Comparando gen T. muris: {tm_protein.id}\n")

    resultados = []
    len_tm = len(tm_protein.seq)
    seq_tm = str(tm_protein.seq)

    for ce_protein in c_elegans_proteins:
        len_ce = len(ce_protein.seq)

        if len_ce / len_tm < min_len_ratio or len_ce / len_tm > max_len_ratio:
            continue

        seq_ce = str(ce_protein.seq)

        alignments = aligner.align(seq_tm, seq_ce)
        alignment = alignments[0]

        # Calcular identidad real: contar matches
        aligned_tm = alignment.aligned[0]
        aligned_ce = alignment.aligned[1]
        matches = 0

        for (start_tm, end_tm), (start_ce, end_ce) in zip(aligned_tm, aligned_ce):
            segment_tm = seq_tm[start_tm:end_tm]
            segment_ce = seq_ce[start_ce:end_ce]
            matches += sum(aa1 == aa2 for aa1, aa2 in zip(segment_tm, segment_ce))

        identity = matches / len_tm

        if identity > 0.7:
            ce_id = ce_protein.id
            ce_in_model = ce_id in c_elegans_model_map
            ce_reactions = ";".join(c_elegans_model_map.get(ce_id, []))

            resultados.append({
                "ID_T_muris": tm_protein.id,
                "ID_C_elegans": ce_id,
                "Score": alignment.score,
                "%_identity": round(identity * 100, 2),
                "In_C_elegans_Model": ce_in_model,
                "C_elegans_Reactions": ce_reactions,
                "Length_T_muris": len_tm,
                "Length_C_elegans": len_ce
            })

    return resultados

# Paralelizar
print("Iniciando alineamientos paralelos...")
start_time = time.time()

resultados_paralelos = Parallel(n_jobs=-1)(
    delayed(comparar_con_todos)(tm_protein) for tm_protein in enzymatic_genes_with_seq
)

orthologs_detailed = list(chain.from_iterable(resultados_paralelos))

print(f"Alineamientos completados en {round((time.time() - start_time) / 60, 2)} minutos.")

Total de genes enzimáticos de T. muris a comparar: 798
Iniciando alineamientos paralelos...
Alineamientos completados en 5.25 minutos.


In [60]:
df_orthologs = pd.DataFrame(orthologs_detailed)
df_orthologs.to_csv("orthologs_with_functions.csv", index=False)
print("Archivo guardado: orthologs_with_functions.csv")

Archivo guardado: orthologs_with_functions.csv


In [61]:
df_orthologs

,ID_T_muris,ID_C_elegans,Score,%_identity,In_C_elegans_Model,C_elegans_Reactions,Length_T_muris,Length_C_elegans
0,TMUE_3000012131,C26E6.4,731.0,81.78,False,,1180,1194
1,TMUE_0000000530,H14A12.2,241.0,74.75,False,,503,501
2,TMUE_3000011916,H14A12.2,245.0,76.91,False,,485,501
3,TMUE_3000012381,F46E10.10,134.0,70.66,False,,334,336
4,TMUE_1000004667,F43G9.1,59.5,72.62,False,,263,358
5,TMUE_3000013077,C03G5.1,286.5,73.26,False,,647,646
6,TMUE_3000013077,C34B2.7,267.5,71.10,False,,647,640
7,TMUE_0000001980,C03G5.1,286.5,73.26,False,,647,646
8,TMUE_0000001980,C34B2.7,267.5,71.10,False,,647,640
9,TMUE_3000013995,LLC1.3a,203.0,71.46,False,,501,495


Comparando gen T. muris: TMUE_2000009066
Comparando gen T. muris: TMUE_3000013113
Comparando gen T. muris: TMUE_3000012046
Comparando gen T. muris: TMUE_2000010394
Comparando gen T. muris: TMUE_3000014089
Comparando gen T. muris: TMUE_2000007657
Comparando gen T. muris: TMUE_2000008564
Comparando gen T. muris: TMUE_1000004140
Comparando gen T. muris: TMUE_3000014695
Comparando gen T. muris: TMUE_3000013824
Comparando gen T. muris: TMUE_2000008193
Comparando gen T. muris: TMUE_3000011643
Comparando gen T. muris: TMUE_2000007226
Comparando gen T. muris: TMUE_3000012821
Comparando gen T. muris: TMUE_2000007570
Comparando gen T. muris: TMUE_3000012081
Comparando gen T. muris: TMUE_3000012968
Comparando gen T. muris: TMUE_1000004841
Comparando gen T. muris: TMUE_0000000356
Comparando gen T. muris: TMUE_2000009037
Comparando gen T. muris: TMUE_3000012033
Comparando gen T. muris: TMUE_2000006971
Comparando gen T. muris: TMUE_2000007334
Comparando gen T. muris: TMUE_0000001119
Comparando gen T